 # Delta Lake tables 
 Use this notebook to explore Delta Lake functionality 


In [21]:
from pyspark.sql.types import StructType, IntegerType, StringType, DoubleType

# define the schema
schema = StructType() \
.add("ProductID", IntegerType(), True) \
.add("ProductName", StringType(), True) \
.add("Category", StringType(), True) \
.add("ListPrice", DoubleType(), True)

df = spark.read.format("csv").option("header","true").schema(schema).load("Files/products/products.csv")
# df now is a Spark DataFrame containing CSV data from "Files/products/products.csv".
display(df)

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 50b4e141-3c06-4e25-97cf-62012e5d9d45)

In [22]:
df.write.format("delta").saveAsTable("managed_products")

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 26, Finished, Available, Finished)

In [23]:
df.write.format("delta").saveAsTable("external_products", path="abfss://5a9cea26-7ad7-4c07-8af2-983eed1d4af7@onelake.dfs.fabric.microsoft.com/41dafd10-a76b-4eef-89dc-33aa8e5c7719/Files/external_products")

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 27, Finished, Available, Finished)

In [24]:
%%sql
DESCRIBE FORMATTED managed_products;

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 28, Finished, Available, Finished)

<Spark SQL result set with 11 rows and 3 fields>

In [95]:
%%sql
DESCRIBE FORMATTED iotdevicedata;

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 101, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 3 fields>

In [26]:
%%sql
DROP TABLE managed_products;
DROP TABLE external_products;

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 31, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [60]:
%%sql
CREATE TABLE products
USING DELTA
LOCATION 'abfss://5a9cea26-7ad7-4c07-8af2-983eed1d4af7@onelake.dfs.fabric.microsoft.com/41dafd10-a76b-4eef-89dc-33aa8e5c7719/Files/external_products';

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 66, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [28]:
%%sql
SELECT * FROM products;

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 33, Finished, Available, Finished)

<Spark SQL result set with 295 rows and 4 fields>

In [29]:
%%sql
UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 34, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [30]:
%%sql
DESCRIBE HISTORY products;

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 35, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 15 fields>

In [31]:
delta_table_path = 'Files/external_products'
# Get the current data
current_data = spark.read.format("delta").load(delta_table_path)
display(current_data)

# Get the version 0 data
original_data = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(original_data)

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e77c5325-bb0a-4a78-b6bb-c2eaf4592f66)

SynapseWidget(Synapse.DataFrame, a1e296cf-5400-43ee-9a0f-90b52a2fbf1b)

In [32]:
%%sql
-- Create a temporary view
CREATE OR REPLACE TEMPORARY VIEW products_view
AS
    SELECT Category, COUNT(*) AS NumProducts, MIN(ListPrice) AS MinPrice, MAX(ListPrice) AS MaxPrice, AVG(ListPrice) AS AvgPrice
    FROM products
    GROUP BY Category;

SELECT *
FROM products_view
ORDER BY Category;    

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 38, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 37 rows and 5 fields>

In [33]:
%%sql
SELECT Category, NumProducts
FROM products_view
ORDER BY NumProducts DESC
LIMIT 10;

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 39, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 2 fields>

In [34]:
from pyspark.sql.functions import col, desc

df_products = spark.sql("SELECT Category, MinPrice, MaxPrice, AvgPrice FROM products_view").orderBy(col("AvgPrice").desc())
display(df_products.limit(6))

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 40, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ab820435-032f-49ee-9864-16fa3ac15d40)

In [87]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "data.txt", device_data, True)

print("Source stream created...")

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 93, Finished, Available, Finished)

Source stream created...


In [88]:
# Write the stream to a delta table
delta_stream_table_path = 'Tables/iotdevicedata'
checkpointpath = 'Files/delta/checkpoint'
# deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
deltaStream = (iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).toTable("iotdevicedata")
)
print("Streaming to delta sink...")

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 94, Finished, Available, Finished)

Streaming to delta sink...


In [89]:
deltaStream.isActive


StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 95, Finished, Available, Finished)

True

In [92]:
%%sql
SELECT * FROM iotdevicedata;

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 98, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [91]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"VANDER"}'''

mssparkutils.fs.put(inputPath + "more-data4.txt", more_data, True)

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 97, Finished, Available, Finished)

True

In [43]:
deltastream.stop()

StatementMeta(, 61205c7b-83bd-4a29-856e-6c18a2e4ef18, 49, Finished, Available, Finished)